<a href="https://colab.research.google.com/github/Brno2305/Pipeline_AWS/blob/main/Pipeline_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importa arquivos do site 'DATA BOSTON' para diretorio local colab**

In [3]:
# Lista de URLs dos arquivos a serem baixados
urls = [
    'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/tmphrybkxuh.csv',
    'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/tmpzxzxeqfb.csv',
    'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/tmpzccn8u4q.csv',
    'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv',
    'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv',
    'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv'
]


In [6]:
import pandas as pd
import requests
from io import StringIO

# Lista para armazenar os DataFrames de cada arquivo
dfs = []

# Loop sobre as URLs
for url in urls:
    # Fazendo solicitação para a URL
    response = requests.get(url)

    # Verifica se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Lê o conteúdo do arquivo CSV em um DataFrame
        data = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(data))

        # Adiciona o DataFrame à lista
        dfs.append(df)
    else:
        print(f'Falha ao baixar {url}. Status code: {response.status_code}')

# Concatena todos os DataFrames em um único DataFrame
df_final = pd.concat(dfs, ignore_index=True)


<ipython-input-6-50d348674f7a>:17: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))
<ipython-input-6-50d348674f7a>:17: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))
<ipython-input-6-50d348674f7a>:17: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))
<ipython-input-6-50d348674f7a>:17: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))
<ipython-input-6-50d348674f7a>:17: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(data))
<ipython-input-6-50d348674f7a>:17: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.


# **Analise dos dados para tratamento posterior(camada bronze)**

In [26]:

# Verificar se há valores nulos
df_final.isnull().sum()

case_enquiry_id                        0
open_dt                                0
sla_target_dt                     210506
closed_dt                         103646
on_time                                0
case_status                            0
closure_reason                         0
case_title                             0
subject                                0
reason                                 0
type                                   0
queue                                  0
department                             0
submitted_photo                        0
closed_photo                           0
location                               0
fire_district                       3004
pwd_district                         776
city_council_district                148
police_district                      666
neighborhood                        1561
neighborhood_services_district       152
ward                                  61
precinct                            1016
location_street_

In [15]:
# Verificar se há valores duplicados no DataFrame
df.duplicated().sum()


0

# **Salva DF local formato .parquet**

In [27]:
# Supondo que df seja o seu DataFrame
df_final.to_parquet('/content/data_boston_services.parquet')


# **Conecta no AWS S3**

In [30]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.2 MB/s eta 0:00:00


In [31]:
import boto3

# Credenciais AWS para acesso
access_key = 'SUA CHAVE DE ACESSO'
secret_key = 'SUA CHAVE SECRETA'
region = 'us-east-1'

# Conectando ao S3
s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region)


## **Move arquivo .parquet para camada BRONZE do S3**

In [32]:
# Nome do seu bucket S3
bucket_name = 'aws-pipeline-py'
file_name = 'data_boston_services.parquet'

# Especifique o caminho do arquivo no seu bucket S3
s3_path = 'bronze/' + file_name

# Envie o arquivo parquet para o S3
s3.upload_file(file_name, bucket_name, s3_path)

# **Trata dados na camada Bronze**

In [2]:
import boto3
import pandas as pd
from io import BytesIO

# Configurações AWS
access_key = 'SUA CHAVE DE ACESSO'
secret_key = 'SUA CHAVE SECRETA'
region = 'us-east-1'
bucket_name = 'aws-pipeline-py'
file_key = 'bronze/data_boston_services.parquet'

# Acessando o arquivo no S3
s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region)
response = s3.get_object(Bucket=bucket_name, Key=file_key)
data = response['Body'].read()

# Carregando o DataFrame
data_buffer = BytesIO(data)
df_bronze_tratado = pd.read_parquet(data_buffer)

# Realizando a limpeza de dados
# Trata alguns campos nulos somente para exemplo:
df_bronze_tratado['on_time'].fillna('N/A', inplace=True)
df_bronze_tratado['case_title'].fillna('N/A', inplace=True)
df_bronze_tratado['submitted_photo'].fillna('N/A', inplace=True)


# **Envia para camada Silver**

In [4]:
# Enviando o DataFrame tratado para a camada 'silver'
bucket_name_silver = 'aws-pipeline-py'
file_key_silver = 'silver/data_boston_services_silver.parquet'

# Convertendo DataFrame para .PARQUET (ou outro formato necessário)
data_silver = df_bronze_tratado.to_parquet(index=False)

# Enviando o DataFrame tratado para o S3
s3.put_object(Body=data_silver, Bucket=bucket_name_silver, Key=file_key_silver)

{'ResponseMetadata': {'RequestId': 'SVZ57BTPBGHY6MBA',
  'HostId': 'x06iScdANv9IaC3kHjaPSKvjcvK+bdp/4YiRd3VOC0m8ODI2ROjcJ6fxk6Uf6NtQBbsbuwEn4WE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'x06iScdANv9IaC3kHjaPSKvjcvK+bdp/4YiRd3VOC0m8ODI2ROjcJ6fxk6Uf6NtQBbsbuwEn4WE=',
   'x-amz-request-id': 'SVZ57BTPBGHY6MBA',
   'date': 'Tue, 30 Jan 2024 03:51:02 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"fea7641aca2575958dffb6054b0c044e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"fea7641aca2575958dffb6054b0c044e"',
 'ServerSideEncryption': 'AES256'}